<a href="https://colab.research.google.com/github/stukgum/Ai-Lyrics-Arsonist-/blob/main/LyricalAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Develop an AI-powered app called "Ai Lyrical Arsonist" that generates lyrics for uploaded beats, provides synchronized playback with visuals, and allows users to record and export their performances. The app should be developed in phases, starting with a core MVP and then enhancing the features.

## Audio Analysis Enhancement

### Subtask:
Integrate a more robust mood detection method into the audio analysis step.

## Lyric Editing

### Subtask:
Implement basic editing features for the generated lyrics.

## Audio Analysis Enhancement

### Subtask:
Integrate a more robust mood detection method into the audio analysis step.

## Beat input handling

### Subtask:
Develop code to accept beat uploads (MP3/WAV) and process links from various sources (YouTube, SoundCloud, RapChat, RapFame, Voloco). This may involve exploring libraries for downloading audio from URLs.


**Reasoning**:
Research and identify appropriate Python libraries for handling audio file uploads and downloading audio from online sources. Based on research, `pydub` is suitable for audio file handling, and `yt-dlp` is effective for downloading from various online sources including YouTube and SoundCloud. Downloading from platforms like RapChat, RapFame, and Voloco might be more complex and may require reverse engineering or dedicated APIs, which are beyond the scope of readily available libraries for general use. Therefore, I will focus on implementing handling for file uploads, YouTube, and SoundCloud using the identified libraries.



## Lyric Editing

### Subtask:
Implement basic editing features for the generated lyrics.

**Reasoning**:
Modify the existing visualizer code to make it more dynamic. Instead of a static plot of the entire audio, we will create a simple animation that updates the soundwave as the audio progresses. This will involve using `matplotlib.animation`.

In [ ]:
# We'll need to reinstall ffmpeg for the animation saving
!apt-get install -y ffmpeg

import matplotlib.pyplot as plt
import numpy as np
import librosa
import matplotlib.animation as animation
import threading
import time
from pydub import AudioSegment
from pydub.playback import play
import os # Import the os module

# Assume processed_audio_path and sr are available from previous steps
# If not, define them for demonstration purposes.
if 'processed_audio_path' not in locals() or not os.path.exists(processed_audio_path):
    processed_audio_path = 'processed_audio/downloaded_audio.wav' # Placeholder
    # Create a dummy file if it doesn't exist for demonstration
    if not os.path.exists('processed_audio'):
        os.makedirs('processed_audio')
    if not os.path.exists(processed_audio_path):
        import soundfile as sf
        samplerate = 22050
        duration = 10
        data = np.zeros(duration * samplerate)
        sf.write(processed_audio_path, data, samplerate)
        print(f"Created a dummy WAV file at: {processed_audio_path}")


# Load the audio file
try:
    y, sr = librosa.load(processed_audio_path)
    audio_duration_s = librosa.get_duration(y=y, sr=sr)
    print(f"Successfully loaded audio file from {processed_audio_path}")
    print(f"Audio duration: {audio_duration_s:.2f} seconds")

except Exception as e:
    print(f"Error loading audio file for visualizer: {e}")
    y, sr = None, None
    audio_duration_s = 0

if y is not None and sr is not None:
    # Calculate RMS energy over small frames
    frame_length = 2048 # You can adjust this
    hop_length = 512   # You can adjust this
    rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
    rms_times = librosa.times_like(rms, sr=sr, hop_length=hop_length)

    # Set up the plot
    fig, ax = plt.subplots(figsize=(12, 4))
    line, = ax.plot([], [], lw=2)
    ax.set_ylim(0, rms.max() * 1.1)
    ax.set_xlim(0, audio_duration_s)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("RMS Energy")
    ax.set_title("Dynamic Soundwave Visualizer")
    plt.close(fig) # Prevent the initial static plot from displaying

    # Function to update the plot for each frame
    def update(frame):
        # 'frame' here will represent the current time in seconds
        # Find the index in rms_times that is closest to the current frame time
        closest_time_index = np.argmin(np.abs(rms_times - frame))

        # Update the data for the line plot up to the current time
        line.set_data(rms_times[:closest_time_index+1], rms[:closest_time_index+1])

        # You might want to adjust the x-axis view window to follow the playback
        # For now, we'll keep the x-axis fixed to show the entire duration.
        # ax.set_xlim(max(0, frame - 5), min(audio_duration_s, frame + 5)) # Example of sliding window

        return line,

    # Function to simulate audio playback and provide current time
    def play_audio_and_sync(audio_path):
        if not os.path.exists(audio_path):
            print(f"Audio file not found for playback: {audio_path}")
            return

        try:
            audio_segment = AudioSegment.from_wav(audio_path)
            audio_duration_ms = len(audio_segment)
            start_time = time.time()
            print("Starting audio playback for synchronization...")
            play(audio_segment) # This is blocking

        except Exception as e:
            print(f"Error during audio playback for synchronization: {e}")


    # Use a shared variable to track elapsed time
    elapsed_time = 0
    is_playing = False

    def update_elapsed_time():
        global elapsed_time, is_playing
        is_playing = True
        start_time = time.time()
        while is_playing and elapsed_time < audio_duration_s:
            elapsed_time = time.time() - start_time
            time.sleep(0.05) # Update every 50 ms

        is_playing = False
        elapsed_time = audio_duration_s # Ensure it reaches the end


    # Create and start the thread to update elapsed time
    time_thread = threading.Thread(target=update_elapsed_time)
    time_thread.start()

    # Create and start the audio playback thread
    audio_playback_thread = threading.Thread(target=play_audio_and_sync, args=(processed_audio_path,))
    audio_playback_thread.start()


    # Function to get the current time for the animation frame
    def get_current_time():
        while elapsed_time < audio_duration_s or is_playing:
            yield elapsed_time
        yield audio_duration_s # Ensure the last frame is shown

    # Create the animation
    # We use frames=get_current_time() to drive the animation by the elapsed time
    ani = animation.FuncAnimation(fig, update, frames=get_current_time, blit=True, interval=50, repeat=False) # interval in ms

    print("Displaying dynamic soundwave visualizer...")
    plt.show()

    # Wait for the audio playback and time threads to finish
    audio_playback_thread.join()
    time_thread.join()
    print("Dynamic visualizer and playback finished.")

else:
    print("Audio data not available, cannot generate dynamic soundwave visualizer.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Created a dummy WAV file at: processed_audio/downloaded_audio.wav


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Successfully loaded audio file from processed_audio/downloaded_audio.wav
Audio duration: 10.00 seconds


/tmp/ipython-input-2084223563.py:52: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(0, rms.max() * 1.1)
/tmp/ipython-input-2084223563.py:124: UserWarning: frames=<function get_current_time at 0x7e4c70991e40> which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, update, frames=get_current_time, blit=True, interval=50, repeat=False) # interval in ms


Starting audio playback for synchronization...
Displaying dynamic soundwave visualizer...
Dynamic visualizer and playback finished.


## Lyric Editing

### Subtask:
Implement basic editing features for the generated lyrics.

**Reasoning**:
I will provide a simple text area where users can view and edit the generated lyrics. This can be implemented using an HTML form within a Flask route, similar to the audio input handling.

In [ ]:
from flask import Flask, request, render_template_string

# Assume 'final_generated_lyrics' is available from the previous step
# If not, use a placeholder for demonstration.
if 'final_generated_lyrics' not in locals() or not final_generated_lyrics:
    final_generated_lyrics = """
    (Verse 1)
    This is a placeholder lyric,
    Ready for you to edit it quick.
    (Chorus)
    Change these lines, make them shine,
    Your creativity, truly divine.
    """
    print("Using placeholder lyrics for the editing step.")

# Simple Flask app setup for lyric editing (can be integrated with the main app later)
app = Flask(__name__)

@app.route('/edit_lyrics', methods=['GET', 'POST'])
def edit_lyrics():
    global final_generated_lyrics # Update the global variable for demonstration
    edited_lyrics = final_generated_lyrics
    if request.method == 'POST':
        edited_lyrics = request.form.get('lyrics_textarea', '')
        # In a real application, you would save the edited_lyrics
        print("Lyrics edited and submitted:")
        print(edited_lyrics)
        # You might want to redirect to another page or update a global variable
        # For now, we just print and the page will show the edited lyrics
        final_generated_lyrics = edited_lyrics


    return render_template_string('''
    <!doctype html>
    <title>Edit Lyrics</title>
    <h1>Edit Your Lyrics</h1>
    <form method=post>
      <textarea name="lyrics_textarea" rows="20" cols="80">{{ lyrics }}</textarea><br><br>
      <input type=submit value=Save>
    </form>
    ''', lyrics=edited_lyrics)

# To run this example Flask app route, you would typically add it to your main Flask app instance
# and run the app.

# Example of how you might run this specific route for testing (requires running the Flask app)
# from werkzeug.serving import run_simple
# run_simple('localhost', 5000, app) # This will run the Flask app and make it accessible

Using placeholder lyrics for the editing step.


**Reasoning**:
The previous code cell defined a Flask route for lyric editing. Now I will provide instructions on how to run the Flask application to access this editing interface. This involves adding the standard Flask run block.

In [ ]:
# To run the Flask app (for demonstration purposes)
# Note: In a Colab environment, you might need to use ngrok or a similar service
# to expose your local Flask server to the internet for testing in a browser.
# For simple testing within Colab, you can use the built-in preview.

if __name__ == '__main__':
    # This will run the Flask development server
    # debug=True allows for automatic reloading and helpful error pages
    # Set host='0.0.0.0' to make the server accessible externally if needed (e.g., for ngrok)
    app.run(debug=True, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that

## Visualizer Enhancement and Editing

### Subtask:
Enhance the soundwave visualizer to be more dynamic and responsive to the audio. Implement basic editing features for the generated lyrics.

In [ ]:
import os
import yt_dlp
from pydub import AudioSegment
from flask import Flask, request, render_template_string, redirect, url_for
from werkzeug.utils import secure_filename

# Define allowed file types for upload
ALLOWED_EXTENSIONS = {'mp3', 'wav'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def process_audio_input(input_source):
    """
    Processes audio input from either a file upload or a URL.

    Args:
        input_source: Either a file storage object (from Flask request.files) or a string URL.

    Returns:
        The path to the processed WAV file, or None if processing failed.
    """
    output_dir = 'processed_audio'
    os.makedirs(output_dir, exist_ok=True)

    if hasattr(input_source, 'filename'):  # Handle file upload
        if input_source and allowed_file(input_source.filename):
            filename = secure_filename(input_source.filename)
            filepath = os.path.join(output_dir, filename)
            input_source.save(filepath)
            try:
                audio = AudioSegment.from_file(filepath)
                wav_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.wav')
                audio.export(wav_path, format='wav')
                os.remove(filepath) # Clean up the original uploaded file
                return wav_path
            except Exception as e:
                print(f"Error processing uploaded audio file: {e}")
                os.remove(filepath)
                return None
        else:
            print("Invalid file type or no file uploaded.")
            return None
    elif isinstance(input_source, str): # Handle URL
        url = input_source
        try:
            ydl_opts = {
                'format': 'bestaudio/best',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'wav',
                }],
                'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
                'extract_flat': True,
                'noplaylist': True,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(url, download=False)
                if 'entries' in info_dict: # Handle playlists, although we set noplaylist=True, some URLs might be interpreted as playlists
                    info_dict = info_dict['entries'][0] # Take the first video
                title = info_dict.get('title', 'downloaded_audio')
                output_filename = os.path.join(output_dir, f"{title}.wav")
                ydl.download([url])
                return output_filename
        except Exception as e:
            print(f"Error downloading audio from URL: {e}")
            return None
    else:
        print("Invalid input source.")
        return None

# Example Flask app setup for demonstration (can be adapted for the final application)
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads' # Not strictly needed with the current logic, but good practice
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

@app.route('/', methods=['GET', 'POST'])
def upload_or_download():
    processed_file = None
    if request.method == 'POST':
        if 'audio_file' in request.files:
            audio_file = request.files['audio_file']
            processed_file = process_audio_input(audio_file)
        elif 'audio_url' in request.form:
            audio_url = request.form['audio_url']
            processed_file = process_audio_input(audio_url)

    return render_template_string('''
    <!doctype html>
    <title>Upload or Download Audio</title>
    <h1>Upload an audio file or provide a URL</h1>
    <form method=post enctype=multipart/form-data>
      <p><input type=file name=audio_file>
         <input type=submit value=Upload>
    </form>
    <form method=post>
      <p><input type=text name=audio_url placeholder="Enter URL">
         <input type=submit value=Download>
    </form>
    {% if processed_file %}
    <h2>Processed Audio:</h2>
    <p>File saved at: {{ processed_file }}</p>
    {% endif %}
    ''', processed_file=processed_file)

# To run this example Flask app, you would typically use:
# if __name__ == '__main__':
#     app.run(debug=True)


ModuleNotFoundError: No module named 'yt_dlp'

In [ ]:
!pip install yt-dlp pydub Flask
!apt-get update
!apt-get install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 81.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,008 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,281 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/u

In [ ]:
import os
import yt_dlp
from pydub import AudioSegment
from flask import Flask, request, render_template_string, redirect, url_for, send_file
from werkzeug.utils import secure_filename
import librosa
import numpy as np
import sounddevice as sd
import scipy.io.wavfile as wav
import time
import threading
from transformers import pipeline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import base64
from io import BytesIO

# Instantiate a single Flask application
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs('processed_audio', exist_ok=True)
os.makedirs('user_recordings', exist_ok=True)
os.makedirs('final_mixes', exist_ok=True)
os.makedirs('visualizer_frames', exist_ok=True) # Directory to save visualizer frames


# Global variables to share data between routes
processed_audio_path = None
audio_analysis_results = None
final_generated_lyrics = None
generator = None # Language model generator instance
audio_duration_s = 0 # To store audio duration for visualization

# Initialize the language model (can be done once at the start)
try:
    # Use a smaller model for potentially faster loading in Colab
    generator = pipeline('text-generation', model='distilgpt2')
    print("Successfully loaded distilgpt2 model for text generation.")
except Exception as e:
    print(f"Error loading language model: {e}")
    generator = None


# Define allowed file types for upload
ALLOWED_EXTENSIONS = {'mp3', 'wav'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def process_audio_input(input_source):
    """
    Processes audio input from either a file upload or a URL.

    Args:
        input_source: Either a file storage object (from Flask request.files) or a string URL.

    Returns:
        The path to the processed WAV file, or None if processing failed.
    """
    output_dir = 'processed_audio'
    os.makedirs(output_dir, exist_ok=True)

    if hasattr(input_source, 'filename'):  # Handle file upload
        if input_source and allowed_file(input_source.filename):
            filename = secure_filename(input_source.filename)
            filepath = os.path.join(output_dir, filename)
            input_source.save(filepath)
            try:
                audio = AudioSegment.from_file(filepath)
                wav_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.wav')
                audio.export(wav_path, format='wav')
                os.remove(filepath) # Clean up the original uploaded file
                return wav_path
            except Exception as e:
                print(f"Error processing uploaded audio file: {e}")
                os.remove(filepath)
                return None
        else:
            print("Invalid file type or no file uploaded.")
            return None
    elif isinstance(input_source, str): # Handle URL
        url = input_source
        try:
            ydl_opts = {
                'format': 'bestaudio/best',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'wav',
                }],
                'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
                'extract_flat': True,
                'noplaylist': True,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(url, download=False)
                if 'entries' in info_dict: # Handle playlists, although we set noplaylist=True, some URLs might be interpreted as playlists
                    info_dict = info_dict['entries'][0] # Take the first video
                title = info_dict.get('title', 'downloaded_audio')
                output_filename = os.path.join(output_dir, f"{title}.wav")
                ydl.download([url])
                return output_filename
        except Exception as e:
            print(f"Error downloading audio from URL: {e}")
            return None
    else:
        print("Invalid input source.")
        return None

def analyze_audio(audio_path):
    """Analyzes audio to detect BPM, tempo, and mood."""
    global audio_duration_s
    if not os.path.exists(audio_path):
        print(f"Audio file not found for analysis: {audio_path}")
        return None

    try:
        y, sr = librosa.load(audio_path)
        audio_duration_s = librosa.get_duration(y=y, sr=sr)
        print(f"Successfully loaded audio file for analysis: {audio_path}")
        print(f"Audio duration: {audio_duration_s:.2f} seconds")


        # Calculate BPM and tempo
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        print(f"Estimated BPM: {tempo:.2f}")

        # Simple heuristic for mood (highly simplified for demonstration)
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        zcr = librosa.feature.zero_crossing_rate(y)[0]
        mean_centroid = np.mean(spectral_centroids)
        mean_zcr = np.mean(zcr)

        estimated_mood = "Neutral"
        if tempo > 120 and mean_centroid > 2000:
            estimated_mood = "Energetic/Happy"
        elif tempo < 80 and mean_centroid < 1500:
             estimated_mood = "Calm/Sad"
        elif tempo > 140 and mean_zcr > 0.1:
             estimated_mood = "Fast/Intense"

        print(f"Estimated Mood (Simplified Heuristic): {estimated_mood}")

        return {
            'bpm': tempo,
            'tempo': tempo,
            'estimated_mood': estimated_mood,
            'duration': audio_duration_s,
            'y': y, # Store audio time series data for visualizer
            'sr': sr  # Store sampling rate for visualizer
        }

    except Exception as e:
        print(f"Error during audio analysis: {e}")
        audio_duration_s = 0
        return None

def create_lyric_prompt(audio_features, user_genre, user_theme, explicit_toggle):
    """
    Creates a prompt string for the language model based on audio features and user inputs,
    with enhanced genre-specific guidance.
    """
    prompt = f"Generate lyrics for a song. "

    genre_instructions = {
        "Hip Hop": "Focus on rhythm, rhyme schemes (like AABB, ABAB), wordplay, and storytelling. Use urban vocabulary. Include ad-libs.",
        "Pop": "Write catchy melodies and simple, relatable themes. Use a verse-chorus structure. Keep language accessible.",
        "Rock": "Incorporate powerful imagery, emotional themes, and potentially a bridge section. Can be introspective or rebellious.",
        "Blues": "Express feelings of sadness, hardship, or sometimes resilience. Use a typical AAB lyrical structure within verses. Include themes of struggle, love, or loss.",
        "Country": "Tell a story, often set in rural or small-town environments. Themes of love, loss, hard work, and everyday life. Use straightforward language and relatable scenarios.",
        "Electronic": "Lyrics can be repetitive or sparse, focusing on mood and atmosphere. Themes might be abstract, futuristic, or focused on the party/dance experience.",
        "R&B": "Focus on smooth melodies and emotional expression, often about love, relationships, or desire. Use vocal runs and ad-libs.",
        "Reggae": "Incorporate themes of peace, love, social justice, and spirituality. Use a relaxed rhythm and potentially Jamaican Patois influences.",
        "Jazz": "Lyrics can be improvisational or tell complex stories. Focus on mood and atmosphere, often with themes of love, city life, or introspection.",
        "Folk": "Tell a story, often with simple melodies and acoustic instrumentation in mind. Themes of nature, history, social issues, or personal journeys.",
    }

    if user_genre and user_genre in genre_instructions:
        prompt += f"The genre is {user_genre}. {genre_instructions[user_genre]} "
    elif user_genre:
        prompt += f"The genre is {user_genre}. Write lyrics appropriate for this genre. "

    if user_theme:
        prompt += f"The theme is about {user_theme}. "

    if audio_features:
        if audio_features.get('bpm') is not None and audio_features['bpm'] > 0:
             if audio_features['bpm'] < 80:
                 prompt += "The beat is slow, the lyrics should have a relaxed flow and longer phrases. "
             elif audio_features['bpm'] < 120:
                 prompt += "The beat has a moderate pace, the lyrics can have a balanced flow. "
             else:
                 prompt += "The beat is fast and energetic, the lyrics should match the rhythm with shorter, punchier lines. "

        if audio_features.get('estimated_mood'):
            prompt += f"The mood of the music is {audio_features['estimated_mood']}. The lyrics should reflect this mood. "

    if not explicit_toggle:
        prompt += "Avoid explicit language and sensitive topics. "
    else:
        prompt += "Explicit language is allowed. "

    prompt += "Write the lyrics now:"

    return prompt

def generate_lyrics(audio_features, user_genre, user_theme, explicit_toggle):
    """Generates lyrics using the language model."""
    global generator
    if generator is None:
        print("Language model not loaded.")
        return "Error: Language model not available."

    if audio_features is None:
        print("Audio analysis results not available.")
        return "Error: Audio analysis results not available."

    prompt = create_lyric_prompt(audio_features, user_genre, user_theme, explicit_toggle)

    try:
        generated_text = generator(
            prompt,
            max_length=500,
            num_return_sequences=1,
            temperature=0.9,
            top_k=50,
            top_p=0.95,
            truncation=True
        )[0]['generated_text']

        # Attempt to remove the prompt from the generated text
        if generated_text.startswith(prompt):
            generated_text = generated_text[len(prompt):].strip()

        return generated_text

    except Exception as e:
        print(f"Error during lyric generation: {e}")
        return f"Error generating lyrics: {e}"

# Note: Recording audio directly in Flask server is not typical or recommended.
# This function is a placeholder/demonstration. Actual recording would be client-side.
def record_audio_dummy(filename, duration):
    """Creates a dummy recording file."""
    print(f"Creating a dummy recording file at {filename} for {duration} seconds.")
    try:
        samplerate = 44100
        data = np.random.rand(int(duration * samplerate)) * 0.1 # Little bit of noise
        wav.write(filename, samplerate, data)
        print(f"Dummy recording saved to {filename}")
        return filename
    except Exception as e:
        print(f"Error creating dummy recording: {e}")
        return None


def combine_audio(beat_path, vocals_path):
    """Combines beat audio with recorded vocals."""
    if not os.path.exists(beat_path):
        return f"Error: Beat audio file not found at {beat_path}"
    if not os.path.exists(vocals_path):
        return f"Error: Vocal audio file not found at {vocals_path}"

    try:
        beat_audio = AudioSegment.from_wav(beat_path)
        vocal_audio = AudioSegment.from_wav(vocals_path)

        # Ensure same frame rate
        if beat_audio.frame_rate != vocal_audio.frame_rate:
            print(f"Warning: Mismatch in frame rates. Resampling vocal audio from {vocal_audio.frame_rate} to {beat_audio.frame_rate}")
            vocal_audio = vocal_audio.set_frame_rate(beat_audio.frame_rate)

        # Ensure similar length
        min_duration = min(len(beat_audio), len(vocal_audio))
        beat_audio = beat_audio[:min_duration]
        vocal_audio = vocal_audio[:min_duration]

        # Overlay
        combined_audio = beat_audio.overlay(vocal_audio)

        export_dir = 'final_mixes'
        os.makedirs(export_dir, exist_ok=True)
        final_output_path = os.path.join(export_dir, 'final_performance_mix.wav')
        combined_audio.export(final_output_path, format="wav")

        return final_output_path

    except Exception as e:
        print(f"Error combining audio files: {e}")
        return f"Error combining audio files: {e}"

def create_soundwave_visualizer(y, sr, audio_duration_s):
    """Generates a static soundwave visualizer image."""
    if y is None or sr is None:
        return None

    try:
        # Calculate RMS energy over small frames
        frame_length = 2048 # You can adjust this
        hop_length = 512   # You can adjust this
        rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
        rms_times = librosa.times_like(rms, sr=sr, hop_length=hop_length)

        # Set up the plot
        fig, ax = plt.subplots(figsize=(12, 4))
        ax.plot(rms_times, rms, lw=2)
        ax.set_ylim(0, rms.max() * 1.1)
        ax.set_xlim(0, audio_duration_s)
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("RMS Energy")
        ax.set_title("Soundwave Visualizer")

        # Save plot to a bytes buffer
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        plt.close(fig) # Close the figure to free memory

        # Encode image to base64
        image_base64 = base64.b64encode(buf.read()).decode('ascii')

        return image_base64

    except Exception as e:
        print(f"Error generating soundwave visualizer: {e}")
        return None


# Flask Routes

@app.route('/', methods=['GET', 'POST'])
def index():
    global processed_audio_path, audio_analysis_results, final_generated_lyrics, audio_duration_s
    processed_file = None
    analysis_results_display = None
    generated_lyrics_display = None
    visualizer_image_base64 = None

    if request.method == 'POST':
        if 'audio_file' in request.files and request.files['audio_file'].filename != '':
            audio_file = request.files['audio_file']
            processed_file = process_audio_input(audio_file)
        elif 'audio_url' in request.form and request.form['audio_url'] != '':
            audio_url = request.form['audio_url']
            processed_file = process_audio_input(audio_url)
        elif 'genre' in request.form: # Handle lyric generation request
             user_genre = request.form.get('genre', 'Hip Hop')
             user_theme = request.form.get('theme', 'general')
             explicit_toggle = request.form.get('explicit', 'off') == 'on'
             if audio_analysis_results:
                 final_generated_lyrics = generate_lyrics(audio_analysis_results, user_genre, user_theme, explicit_toggle)
                 generated_lyrics_display = final_generated_lyrics
             else:
                 generated_lyrics_display = "Please upload/process audio first to generate lyrics."

        if processed_file:
            processed_audio_path = processed_file
            audio_analysis_results = analyze_audio(processed_audio_path)
            if audio_analysis_results:
                analysis_results_display = f"BPM: {audio_analysis_results['bpm']:.2f}, Mood: {audio_analysis_results['estimated_mood']}, Duration: {audio_analysis_results['duration']:.2f} seconds"
                # Generate visualizer image immediately after analysis
                visualizer_image_base64 = create_soundwave_visualizer(audio_analysis_results.get('y'), audio_analysis_results.get('sr'), audio_analysis_results.get('duration', 0))


    # Display existing data on GET requests or after POST
    if audio_analysis_results:
         analysis_results_display = f"BPM: {audio_analysis_results['bpm']:.2f}, Mood: {audio_analysis_results['estimated_mood']}, Duration: {audio_analysis_results['duration']:.2f} seconds"
         # Regenerate visualizer on page load if analysis results exist
         visualizer_image_base64 = create_soundwave_visualizer(audio_analysis_results.get('y'), audio_analysis_results.get('sr'), audio_analysis_results.get('duration', 0))

    if final_generated_lyrics:
        generated_lyrics_display = final_generated_lyrics


    return render_template_string('''
    <!doctype html>
    <title>Ai Lyrical Arsonist</title>
    <h1>Ai Lyrical Arsonist</h1>

    <h2>Upload an audio file or provide a URL</h2>
    <form method=post enctype=multipart/form-data>
      <p><input type=file name=audio_file>
         <input type=submit value=Upload File>
    </form>
    <form method=post>
      <p><input type=text name=audio_url placeholder="Enter URL">
         <input type=submit value=Download URL>
    </form>

    {% if processed_audio_path %}
    <h2>Processed Audio:</h2>
    <p>File ready for analysis and lyric generation.</p>
    {% endif %}

    {% if analysis_results_display %}
    <h2>Audio Analysis Results:</h2>
    <p>{{ analysis_results_display }}</p>
    {% endif %}

    {% if visualizer_image_base64 %}
    <h2>Soundwave Visualizer:</h2>
    <img src="data:image/png;base64,{{ visualizer_image_base64 }}" alt="Soundwave Visualizer">
    {% endif %}


    <h2>Generate Lyrics</h2>
    <form method=post>
      <p>Genre: <input type=text name=genre value="Hip Hop"></p>
      <p>Theme: <input type=text name=theme value="general"></p>
      <p>Explicit: <input type=checkbox name=explicit value="on"></p>
      <input type=submit value=Generate Lyrics>
    </form>

    {% if generated_lyrics_display %}
    <h2>Generated Lyrics:</h2>
    <pre>{{ generated_lyrics_display }}</pre>
    <p><a href="{{ url_for('edit_lyrics') }}">Edit Lyrics</a></p>
    {% endif %}

    {% if processed_audio_path and generated_lyrics_display %}
    <h2>Playback and Recording</h2>
    <p><a href="{{ url_for('playback') }}">Start Playback and Sync (Text-based simulation)</a></p>
     <p><a href="{{ url_for('record_page') }}">Record Performance (Client-side required for real app)</a></p>
    {% endif %}

    {% if processed_audio_path %}
        {% set dummy_recording_path = 'user_recordings/user_performance.wav' %}
        {% if os.path.exists(dummy_recording_path) %}
            <h2>Combine Audio</h2>
            <p><a href="{{ url_for('combine') }}">Combine Beat and Vocals</a></p>
        {% else %}
             <h2>Record Performance</h2>
             <p>Record your performance first to enable audio combining.</p>
              <p><a href="{{ url_for('record_page') }}">Record Performance (Client-side required for real app)</a></p>
        {% endif %}
    {% endif %}


    {% if os.path.exists('final_mixes/final_performance_mix.wav') %}
    <h2>Final Mix</h2>
    <p><a href="{{ url_for('download_mix') }}">Download Final Mix</a></p>
    {% endif %}


    ''', processed_audio_path=processed_audio_path,
         analysis_results_display=analysis_analysis_results,
         generated_lyrics_display=generated_lyrics_display,
         visualizer_image_base64=visualizer_image_base64,
         os=os) # Pass os to check file existence

@app.route('/edit_lyrics', methods=['GET', 'POST'])
def edit_lyrics():
    global final_generated_lyrics
    if request.method == 'POST':
        final_generated_lyrics = request.form.get('lyrics_textarea', '')
        print("Lyrics edited and saved.")
        return redirect(url_for('index')) # Redirect back to the main page

    return render_template_string('''
    <!doctype html>
    <title>Edit Lyrics</title>
    <h1>Edit Your Lyrics</h1>
    <form method=post>
      <textarea name="lyrics_textarea" rows="20" cols="80">{{ lyrics }}</textarea><br><br>
      <input type=submit value=Save>
    </form>
    <p><a href="{{ url_for('index') }}">Back to Home</a></p>
    ''', lyrics=final_generated_lyrics if final_generated_lyrics else "No lyrics generated yet.")


@app.route('/playback')
def playback():
    global processed_audio_path, final_generated_lyrics

    if not processed_audio_path or not os.path.exists(processed_audio_path):
        return "Error: Processed audio not available.", 400
    if not final_generated_lyrics:
        return "Error: Lyrics not generated yet.", 400

    # Simple lyric synchronization strategy (as before)
    lyrics_lines = final_generated_lyrics.strip().split('\n')
    try:
        audio_segment = AudioSegment.from_wav(processed_audio_path)
        audio_duration_ms = len(audio_segment)
        num_lines = len(lyrics_lines)
        line_duration_ms = audio_duration_ms / num_lines if num_lines > 0 else 0
    except Exception as e:
        return f"Error loading audio for playback: {e}", 500

    # This is a simplified text-based playback simulation for the notebook environment.
    # A real web app would require client-side JavaScript for synchronized audio and lyric display.
    playback_output = "Starting playback and lyric sync...\n"
    for i, line in enumerate(lyrics_lines):
        playback_output += line + "\n"
        if i < len(lyrics_lines) - 1:
             # Simulate timing (not real-time in a Flask response)
             pass # In a real app, this timing is handled client-side

    playback_output += "Playback and lyric display completed.\n"

    return render_template_string('''
    <!doctype html>
    <title>Playback and Sync</title>
    <h1>Playback and Lyric Sync (Text-based simulation)</h1>
    <pre>{{ output }}</pre>
    <p><a href="{{ url_for('index') }}">Back to Home</a></p>
    ''', output=playback_output)


@app.route('/record_page')
def record_page():
     # This route provides a page with instructions for recording.
     # Actual recording is client-side in a real web app.
     # For this demo, we'll create a dummy recording file when the user clicks 'Record Dummy'.
     return render_template_string('''
     <!doctype html>
     <title>Record Performance</title>
     <h1>Record Your Performance</h1>
     <p>In a real web application, this page would use your microphone to record your vocals over the beat.</p>
     <p>For this demonstration, you can create a dummy recording file to proceed with the combining step.</p>
     <p><a href="{{ url_for('create_dummy_recording') }}">Create Dummy Recording File</a></p>
     <p><a href="{{ url_for('index') }}">Back to Home</a></p>
     ''')

@app.route('/create_dummy_recording')
def create_dummy_recording():
    global audio_duration_s
    dummy_recording_path = 'user_recordings/user_performance.wav'
    # Create a dummy recording with the same duration as the processed audio
    record_audio_dummy(dummy_recording_path, audio_duration_s if audio_duration_s > 0 else 10) # Default to 10 secs if duration is 0
    return redirect(url_for('index'))


@app.route('/combine')
def combine():
    global processed_audio_path
    user_recording_path = 'user_recordings/user_performance.wav' # Assume this is the default recording file

    if not processed_audio_path or not os.path.exists(processed_audio_path):
        return "Error: Processed beat audio not available.", 400
    if not os.path.exists(user_recording_path):
        return f"Error: User recording not found at {user_recording_path}. Please ensure a recording has been made (or a dummy file exists).", 400

    final_mix_path = combine_audio(processed_audio_path, user_recording_path)

    if final_mix_path.startswith("Error"):
        return final_mix_path, 500
    else:
        return render_template_string('''
        <!doctype html>
        <title>Combine Audio</title>
        <h1>Audio Combined!</h1>
        <p>Final mix saved to: {{ final_mix_path }}</p>
        <p><a href="{{ url_for('download_mix') }}">Download Final Mix</a></p>
        <p><a href="{{ url_for('index') }}">Back to Home</a></p>
        ''', final_mix_path=final_mix_path)

@app.route('/download_mix')
def download_mix():
    final_mix_path = 'final_mixes/final_performance_mix.wav'
    if os.path.exists(final_mix_path):
        return send_file(final_mix_path, as_attachment=True)
    else:
        return "Error: Final mix file not found.", 404


# To run the Flask app (for demonstration purposes)
# Note: In a Colab environment, you might need to use ngrok or a similar service
# to expose your local Flask server to the internet for testing in a browser.
# For simple testing within Colab, you can use the built-in preview.

if __name__ == '__main__':
    # This will run the Flask development server
    # debug=True allows for automatic reloading and helpful error pages
    # Set host='0.0.0.0' to make the server accessible externally if needed (e.e., for ngrok)
    app.run(debug=True, host='0.0.0.0', port=5000)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


Successfully loaded distilgpt2 model for text generation.
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [ ]:
# Generate lyrics with a different genre and theme using the updated create_lyric_prompt function

user_genre_example_2 = "Hip Hop"
user_theme_example_2 = "overcoming challenges"
explicit_toggle_example_2 = True

if audio_analysis_results is not None and generator is not None:
    # Create the prompt with the new genre and theme
    lyric_prompt_example_2 = create_lyric_prompt(audio_analysis_results, user_genre_example_2, user_theme_example_2, explicit_toggle_example_2)
    print(f"\nGenerated Prompt (Hip Hop Example):\n{lyric_prompt_example_2}")

    try:
        # Generate lyrics using the language model
        generated_lyrics_example_2 = generator(
            lyric_prompt_example_2,
            max_length=300,  # Adjust max_length as needed
            num_return_sequences=1,
            temperature=0.9,
            top_k=50,
            top_p=0.95,
            truncation=True
        )[0]['generated_text']

        # Attempt to remove the prompt from the generated text
        if generated_lyrics_example_2.startswith(lyric_prompt_example_2):
            generated_lyrics_example_2 = generated_lyrics_example_2[len(lyric_prompt_example_2):].strip()


        print("\nGenerated Lyrics (Hip Hop Example):")
        print(generated_lyrics_example_2)

    except Exception as e:
        print(f"Error during Hip Hop lyric generation: {e}")
else:
    print("Audio analysis results or language model not available, cannot generate Hip Hop lyrics.")

In [ ]:
# Refine the lyric prompt and generate lyrics using the model

# Using the example user inputs from the previous step
user_genre_example = "Blues"
user_theme_example = "hard times"
explicit_toggle_example = False

if audio_analysis_results is not None and generator is not None:
    # Create the refined prompt
    refined_lyric_prompt = create_lyric_prompt(audio_analysis_results, user_genre_example, user_theme_example, explicit_toggle_example)
    print(f"\nUsing Refined Prompt:\n{refined_lyric_prompt}")

    try:
        # Generate lyrics using the language model with the refined prompt
        # Adjust max_length and other parameters as needed for desired lyric length and style
        generated_lyrics_refined = generator(
            refined_lyric_prompt,
            max_length=500,  # Increased max_length
            num_return_sequences=1,
            temperature=0.9, # Increased temperature for more creativity
            top_k=50,       # Added top_k sampling
            top_p=0.95,     # Added top_p (nucleus) sampling
            truncation=True
        )[0]['generated_text']

        # Attempt to remove the prompt from the generated text
        if generated_lyrics_refined.startswith(refined_lyric_prompt):
            generated_lyrics_refined = generated_lyrics_refined[len(refined_lyric_prompt):].strip()

        print("\nGenerated Lyrics (Refined):")
        print(generated_lyrics_refined)

        # You can store this in a variable for later use, e.g., for the playback and sync step
        final_generated_lyrics = generated_lyrics_refined

    except Exception as e:
        print(f"Error during refined lyric generation: {e}")
        final_generated_lyrics = None
else:
    print("Audio analysis results or language model not available, cannot generate refined lyrics.")

**Reasoning**:
The previous command failed because the `yt-dlp` library was not installed. I need to install `yt-dlp` and its dependencies, including `ffmpeg` which is required by `pydub` for audio format conversion and by `yt-dlp` for post-processing.



In [ ]:
!pip install yt-dlp pydub Flask
!apt-get update
!apt-get install -y ffmpeg

**Reasoning**:
The necessary libraries and ffmpeg have been installed. Now I can re-run the code to define the `process_audio_input` function and the basic Flask app setup, which addresses steps 3, 4, and 5 of the instructions.



In [ ]:
import os
import yt_dlp
from pydub import AudioSegment
from flask import Flask, request, render_template_string, redirect, url_for
from werkzeug.utils import secure_filename

# Define allowed file types for upload
ALLOWED_EXTENSIONS = {'mp3', 'wav'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def process_audio_input(input_source):
    """
    Processes audio input from either a file upload or a URL.

    Args:
        input_source: Either a file storage object (from Flask request.files) or a string URL.

    Returns:
        The path to the processed WAV file, or None if processing failed.
    """
    output_dir = 'processed_audio'
    os.makedirs(output_dir, exist_ok=True)

    if hasattr(input_source, 'filename'):  # Handle file upload
        if input_source and allowed_file(input_source.filename):
            filename = secure_filename(input_source.filename)
            filepath = os.path.join(output_dir, filename)
            input_source.save(filepath)
            try:
                audio = AudioSegment.from_file(filepath)
                wav_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.wav')
                audio.export(wav_path, format='wav')
                os.remove(filepath) # Clean up the original uploaded file
                return wav_path
            except Exception as e:
                print(f"Error processing uploaded audio file: {e}")
                os.remove(filepath)
                return None
        else:
            print("Invalid file type or no file uploaded.")
            return None
    elif isinstance(input_source, str): # Handle URL
        url = input_source
        try:
            ydl_opts = {
                'format': 'bestaudio/best',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'wav',
                }],
                'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
                'extract_flat': True,
                'noplaylist': True,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(url, download=False)
                if 'entries' in info_dict: # Handle playlists, although we set noplaylist=True, some URLs might be interpreted as playlists
                    info_dict = info_dict['entries'][0] # Take the first video
                title = info_dict.get('title', 'downloaded_audio')
                output_filename = os.path.join(output_dir, f"{title}.wav")
                ydl.download([url])
                return output_filename
        except Exception as e:
            print(f"Error downloading audio from URL: {e}")
            return None
    else:
        print("Invalid input source.")
        return None

# Example Flask app setup for demonstration (can be adapted for the final application)
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads' # Not strictly needed with the current logic, but good practice
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

@app.route('/', methods=['GET', 'POST'])
def upload_or_download():
    processed_file = None
    if request.method == 'POST':
        if 'audio_file' in request.files:
            audio_file = request.files['audio_file']
            processed_file = process_audio_input(audio_file)
        elif 'audio_url' in request.form:
            audio_url = request.form['audio_url']
            processed_file = process_audio_input(audio_url)

    return render_template_string('''
    <!doctype html>
    <title>Upload or Download Audio</title>
    <h1>Upload an audio file or provide a URL</h1>
    <form method=post enctype=multipart/form-data>
      <p><input type=file name=audio_file>
         <input type=submit value=Upload>
    </form>
    <form method=post>
      <p><input type=text name=audio_url placeholder="Enter URL">
         <input type=submit value=Download>
    </form>
    {% if processed_file %}
    <h2>Processed Audio:</h2>
    <p>File saved at: {{ processed_file }}</p>
    {% endif %}
    ''', processed_file=processed_file)

# To run this example Flask app, you would typically use:
# if __name__ == '__main__':
#     app.run(debug=True)

## Audio analysis

### Subtask:
Implement audio analysis to detect BPM, tempo, and mood. We can use libraries like Librosa for this purpose.


**Reasoning**:
Import the necessary libraries for audio analysis and load the processed audio file.



In [ ]:
import librosa
import numpy as np
import os

# Assuming the previous step saved the processed WAV file to 'processed_audio/downloaded_audio.wav' or similar
# We need a placeholder filename for demonstration purposes.
# In a real application, you would get the actual filename from the previous step's output.
processed_audio_path = 'processed_audio/downloaded_audio.wav' # Replace with actual path

# Create a dummy file if it doesn't exist for demonstration
if not os.path.exists('processed_audio'):
    os.makedirs('processed_audio')
if not os.path.exists(processed_audio_path):
    # Create a silent WAV file for demonstration if the real one doesn't exist
    import soundfile as sf
    samplerate = 22050
    duration = 10
    data = np.zeros(duration * samplerate)
    sf.write(processed_audio_path, data, samplerate)
    print(f"Created a dummy WAV file at: {processed_audio_path}")


try:
    y, sr = librosa.load(processed_audio_path)
    print(f"Successfully loaded audio file: {processed_audio_path}")
    print(f"Audio time series shape: {y.shape}")
    print(f"Sampling rate: {sr}")

except Exception as e:
    print(f"Error loading audio file: {e}")
    y, sr = None, None # Set to None if loading fails


**Reasoning**:
Calculate the BPM and tempo using librosa's beat tracking function and explore potential mood detection methods.



In [ ]:
# Calculate BPM and tempo
if y is not None and sr is not None:
    try:
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        print(f"Estimated BPM: {tempo:.2f}")
        # beat_frames contains the frame indices of the detected beats
        # To get the time of the beats: librosa.frames_to_time(beat_frames, sr=sr)

        # Mood detection - Exploring methods
        # Librosa itself doesn't have a direct 'mood' function.
        # Mood is a complex concept often derived from a combination of features.
        # Potential features to analyze for mood:
        # 1. Spectral Centroid: Indicates the 'brightness' of the sound. Higher values can suggest happier/brighter moods.
        # 2. Spectral Rolloff: Indicates the shape of the spectrum. Can relate to the 'darkness' or 'brightness'.
        # 3. Zero-Crossing Rate: How often the signal crosses the zero axis. Higher values can indicate more percussive or energetic sounds.
        # 4. Mel-Frequency Cepstral Coefficients (MFCCs): Capture the timbre of the sound. Can be used as features for a mood classification model.
        # 5. Chroma Features: Represent the spectral energy across different pitch classes. Can be related to harmony and tonality, which influence mood.
        # 6. Tempo and Rhythm features (already calculated): Fast tempo often relates to energetic/happy moods, slow tempo to calm/sad moods.
        # 7. Dynamic Range/Loudness: Can relate to intensity and energy.

        # For a simple estimation without a pre-trained model, we can combine a few features.
        # A very basic approach could be to look at tempo and spectral features.
        # Let's calculate spectral centroid and zero-crossing rate as examples.

        # Calculate spectral centroid
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        # Calculate zero-crossing rate
        zcr = librosa.feature.zero_crossing_rate(y)[0]

        # Simple heuristic for mood (highly simplified for demonstration)
        # This is NOT a robust mood detection method but illustrates using features.
        # A real-world application would use a trained machine learning model.
        mean_centroid = np.mean(spectral_centroids)
        mean_zcr = np.mean(zcr)

        estimated_mood = "Neutral" # Default mood
        if tempo > 120 and mean_centroid > 2000: # Higher tempo and brighter sound
            estimated_mood = "Energetic/Happy"
        elif tempo < 80 and mean_centroid < 1500: # Lower tempo and darker sound
             estimated_mood = "Calm/Sad"
        elif tempo > 140 and mean_zcr > 0.1: # Very high tempo and high ZCR
             estimated_mood = "Fast/Intense"

        print(f"Estimated Mood (Simplified Heuristic): {estimated_mood}")

        # Store the results
        audio_analysis_results = {
            'bpm': tempo,
            'tempo': tempo, # BPM and tempo are often used interchangeably in this context
            'estimated_mood': estimated_mood
        }
        print("Audio analysis results stored.")

    except Exception as e:
        print(f"Error during audio analysis: {e}")
        audio_analysis_results = None # Set to None if analysis fails
else:
    print("Audio data not loaded, skipping analysis.")
    audio_analysis_results = None


## Ai lyrics generation

### Subtask:
Set up an AI model to generate lyrics based on the analyzed audio features (BPM, tempo, mood) and user-specified parameters (genre, theme, explicit toggle). This will involve choosing and integrating a suitable language model.


**Reasoning**:
Choose a suitable language model and integrate it with the audio analysis results and user inputs to generate lyrics. I will use the `transformers` library to interact with a pre-trained language model suitable for text generation.



In [ ]:
from transformers import pipelinem

# Choose a suitable language modelm
# 'gpt2' is a good starting point for text generation due to its availability and capability.
# For more creative or longer lyrics, larger models might be considered, but require more resources.
# We'll use a text generation pipeline for ease of use.
try:
    generator = pipeline('text-generation', model='gpt2')
    print("Successfully loaded GPT-2 model for text generation.")
except Exception as e:
    print(f"Error loading language model: {e}")
    generator = None

# Design the prompt structure
# The prompt should incorporate the audio features and user preferences.
# We'll create a function to generate the prompt string.
def create_lyric_prompt(audio_features, user_genre, user_theme, explicit_toggle):
    """
    Creates a prompt string for the language model based on audio features and user inputs.

    Args:
        audio_features: Dictionary containing 'bpm', 'tempo', 'estimated_mood'.
        user_genre: User-specified music genre (e.g., "Hip Hop", "Pop", "Rock").
        user_theme: User-specified theme for the lyrics (e.g., "love", "party", "struggle").
        explicit_toggle: Boolean indicating whether explicit lyrics are allowed.

    Returns:
        A formatted prompt string.
    """
    prompt = f"Generate lyrics for a song. "

    if user_genre:
        prompt += f"The genre is {user_genre}. "
    if user_theme:
        prompt += f"The theme is about {user_theme}. "

    # Incorporate audio features (BPM and Mood are most relevant for lyrical flow and content)
    if audio_features:
        if audio_features.get('bpm') is not None and audio_features['bpm'] > 0:
             # Adjusting prompt based on BPM for rhythm suggestion
             if audio_features['bpm'] < 80:
                 prompt += "The beat is slow, the lyrics should have a relaxed flow. "
             elif audio_features['bpm'] < 120:
                 prompt += "The beat has a moderate pace. "
             else:
                 prompt += "The beat is fast and energetic, the lyrics should match the rhythm. "

        if audio_features.get('estimated_mood'):
            prompt += f"The mood of the music is {audio_features['estimated_mood']}. "

    # Add explicit toggle consideration (simple instruction for the model)
    if not explicit_toggle:
        prompt += "Avoid explicit language. "

    prompt += "Here are the lyrics:" # Indicate where the generated lyrics should start

    return prompt

# Example usage of the prompt creation function
# Assuming some placeholder user inputs for demonstration
user_genre = "Hip Hop"
user_theme = "overcoming challenges"
explicit_toggle = True # Allow explicit lyrics

if audio_analysis_results is not None:
    lyric_prompt = create_lyric_prompt(audio_analysis_results, user_genre, user_theme, explicit_toggle)
    print(f"\nGenerated Prompt:\n{lyric_prompt}")

    # Generate lyrics using the model
    if generator is not None:
        try:
            # Generate a sequence of tokens. max_length controls the length of the generated text.
            # num_return_sequences can generate multiple options.
            # temperature can control randomness (higher temperature = more creative/random).
            # top_k and top_p can control the sampling strategy.
            # We'll use a basic generation for now.
            generated_lyrics = generator(lyric_prompt, max_length=150, num_return_sequences=1, temperature=0.7)[0]['generated_text']

            # The model might include the prompt in the output, so we can try to remove it.
            if generated_lyrics.startswith(lyric_prompt):
                generated_lyrics = generated_lyrics[len(lyric_prompt):].strip()

            print("\nGenerated Lyrics:")
            print(generated_lyrics)

        except Exception as e:
            print(f"Error during lyric generation: {e}")
    else:
        print("Language model not loaded, cannot generate lyrics.")
else:
    print("Audio analysis results not available, cannot create lyric prompt.")


## Lyric Editing

### Subtask:
Implement basic editing features for the generated lyrics.

**Reasoning**:
Implement basic editing features for the generated lyrics using `ipywidgets` to create an interactive text area within the notebook.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Assume 'final_generated_lyrics' is available from the previous step
# If not, use a placeholder for demonstration.
if 'final_generated_lyrics' not in locals() or not final_generated_lyrics:
    final_generated_lyrics = """
    (Verse 1)
    This is a placeholder lyric,
    Ready for you to edit it quick.
    (Chorus)
    Change these lines, make them shine,
    Your creativity, truly divine.
    """
    print("Using placeholder lyrics for the editing step.")

# Create a Textarea widget for editing
lyrics_textarea = widgets.Textarea(
    value=final_generated_lyrics,
    description='Edit Lyrics:',
    disabled=False,
    layout={'width': '80%', 'height': '300px'}
)

# Create a button to simulate saving the edited lyrics
save_button = widgets.Button(
    description='Save Lyrics',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to save the edited lyrics'
)

output_area = widgets.Output()

def on_save_button_clicked(b):
    with output_area:
        output_area.clear_output()
        global final_generated_lyrics
        final_generated_lyrics = lyrics_textarea.value
        print("Lyrics saved:")
        print(final_generated_lyrics)
        # In a real application, you would save the edited_lyrics to a file or database

save_button.on_click(on_save_button_clicked)

print("Edit the lyrics in the text area below and click 'Save Lyrics'.")
display(lyrics_textarea, save_button, output_area)

**Reasoning**:
The previous steps failed because of missing audio libraries (`PortAudio` and `libsndfile`). Install these libraries using `apt-get`.

In [ ]:
!apt-get update
!apt-get install -y libportaudio2 libsndfile1

## Recording and Export

### Subtask:
Implement audio recording and export functionality to capture user performances and create a final output file.

**Reasoning**:
Install necessary libraries for audio recording and processing (`sounddevice`, `numpy`, `scipy`, and ensure `ffmpeg` is installed).

In [ ]:
!pip install sounddevice numpy scipy
!apt-get update && apt-get install -y ffmpeg

**Reasoning**:
Develop code to record audio from the user's microphone. This will use `sounddevice` to capture input for a specified duration.

In [ ]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import os
import threading
import time

# Define recording parameters
samplerate = 44100  # Standard audio CD quality
duration = 10       # seconds (you can adjust this or make it dynamic)
# Ensure the output directory exists
output_dir = 'user_recordings'
os.makedirs(output_dir, exist_ok=True)
output_filename = os.path.join(output_dir, 'user_performance.wav')

print(f"Preparing to record for {duration} seconds...")
print("Speak into your microphone when prompted.")

# Function to handle the recording
def record_audio(filename, samplerate, duration):
    print("Recording...")
    myrecording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='float32')
    sd.wait()  # Wait until recording is finished
    wav.write(filename, samplerate, myrecording)
    print(f"Recording saved to {filename}")

# Note: In a real application within a web browser, you would use Web Audio API
# for microphone access and recording, as direct access via Python in a notebook
# might have limitations depending on the environment. This is a basic demonstration.

# You would typically trigger this function when the user clicks a 'Record' button.
# For demonstration, we'll call it directly.

# record_audio(output_filename, samplerate, duration) # Uncomment to run recording
# print("Recording code is defined. Uncomment the function call to run.")

**Reasoning**:
Develop code to combine the recorded user audio with the original beat audio using `pydub` (which requires `ffmpeg`). This involves loading both audio files, overlaying them, and exporting the combined audio.

In [ ]:
from pydub import AudioSegment
import os

# Assuming processed_audio_path and output_filename (user recording) are available
# If not, use placeholders for demonstration.
if 'processed_audio_path' not in locals() or not os.path.exists(processed_audio_path):
     processed_audio_path = 'processed_audio/downloaded_audio.wav'
     # Create dummy if it doesn't exist (should be handled by earlier steps)
     if not os.path.exists(processed_audio_path):
        import soundfile as sf
        samplerate = 22050
        duration = 10
        data = np.zeros(duration * samplerate)
        sf.write(processed_audio_path, data, samplerate)
        print(f"Created a dummy processed_audio.wav at: {processed_audio_path}")

if 'output_filename' not in locals() or not os.path.exists(output_filename):
     output_filename = 'user_recordings/user_performance.wav'
     # Create dummy if it doesn't exist (you would replace this with an actual recording)
     if not os.path.exists(output_filename):
         import soundfile as sf
         samplerate = 44100
         duration = 10
         data = np.random.rand(duration * samplerate) * 0.1 # Little bit of noise
         sf.write(output_filename, data, samplerate)
         print(f"Created a dummy user_performance.wav at: {output_filename}")


try:
    # Load the beat and the recorded vocals
    beat_audio = AudioSegment.from_wav(processed_audio_path)
    vocal_audio = AudioSegment.from_wav(output_filename)

    # Ensure both audio segments have the same frame rate (important for overlaying)
    # If they don't, resample one to match the other.
    if beat_audio.frame_rate != vocal_audio.frame_rate:
        print(f"Warning: Mismatch in frame rates. Resampling vocal audio from {vocal_audio.frame_rate} to {beat_audio.frame_rate}")
        vocal_audio = vocal_audio.set_frame_rate(beat_audio.frame_rate)


    # Ensure both audio segments are of similar length for simple overlay
    # In a real app, you'd likely trim or loop the beat, or handle different lengths based on performance.
    min_duration = min(len(beat_audio), len(vocal_audio))
    beat_audio = beat_audio[:min_duration]
    vocal_audio = vocal_audio[:min_duration]


    # Overlay the vocal track onto the beat track
    # You might need to adjust the volume of one track relative to the other for a good mix
    # For example: beat_audio.overlay(vocal_audio - 5) # Reduce vocal volume by 5 dB
    combined_audio = beat_audio.overlay(vocal_audio)

    # Export the combined audio
    export_dir = 'final_mixes'
    os.makedirs(export_dir, exist_ok=True)
    final_output_path = os.path.join(export_dir, 'final_performance_mix.wav')
    combined_audio.export(final_output_path, format="wav")

    print(f"Combined audio exported to: {final_output_path}")

except Exception as e:
    print(f"Error combining audio files: {e}")
    print("Please ensure both processed_audio.wav and user_performance.wav exist and are valid audio files.")

**Reasoning**:
(Optional, but part of a full export feature) Develop code to combine the mixed audio with the visualizer (and potentially synchronized lyrics as a subtitle track or burned-in text) into a single video file using `ffmpeg`. This is a more complex step and might require exploring `ffmpeg` command-line arguments or Python wrappers.

In [ ]:
import subprocess
import os

# This is a more advanced step requiring careful use of ffmpeg.
# Combining audio, visualizer animation (if dynamic), and lyrics into video
# would typically involve:
# 1. Generating the dynamic visualizer frames as images.
# 2. Generating a subtitle file (like .srt) for the synchronized lyrics.
# 3. Using ffmpeg to combine the beat+vocal audio, visualizer images, and subtitles into an MP4 or other video format.

# Example of a basic ffmpeg command to combine audio and a static image into a video:
# subprocess.run(['ffmpeg', '-loop', '1', '-i', 'static_visualizer.png', '-i', 'final_mixes/final_performance_mix.wav', '-c:v', 'libx264', '-tune', 'stillimage', '-c:a', 'aac', '-b:a', '192k', '-pix_fmt', 'yuv420p', '-shortest', 'final_output.mp4'])

# A dynamic visualizer and synchronized lyrics would significantly increase complexity.
# For now, we'll focus on the audio mix. Full video export can be a later enhancement.

print("Video export functionality (combining audio, visualizer, lyrics) is complex and would require generating video frames and potentially subtitle files.")
print("The code for audio mixing is provided in the previous cell.")
print("Full video export can be explored as a further enhancement.")

## Playback and lyric sync

### Subtask:
Develop a system to play the uploaded beat and display the generated lyrics in a karaoke-style synchronized manner.


**Reasoning**:
Import necessary libraries for audio playback and lyric synchronization.



In [ ]:
import time
import threading
import os
from pydub import AudioSegment
from pydub.playback import play

# Assuming the processed audio path from the previous step
processed_audio_path = 'processed_audio/downloaded_audio.wav'

# Assuming 'final_generated_lyrics' is available from the previous subtask as a string
# Use the refined lyrics if available, otherwise use the dummy
if 'final_generated_lyrics' in locals() and final_generated_lyrics:
    generated_lyrics_to_sync = final_generated_lyrics
    print("Using refined generated_lyrics for synchronization.")
elif 'generated_lyrics' in locals() and generated_lyrics:
     generated_lyrics_to_sync = generated_lyrics
     print("Using initial generated_lyrics for synchronization.")
else:
    generated_lyrics_to_sync = """
    (Verse 1)
    Yo, the beat is slow, feeling the flow,
    Sun going down, putting on a show.
    Got that calm vibe, nothing to hide,
    Just chilling here, with the worldwide tide.

    (Chorus)
    Calm and sad, but not really bad,
    Just reflecting on the times we had.
    The tempo's low, letting emotions show,
    In this mellow groove, watch the feelings grow.

    (Verse 2)
    Streetlights are dim, thoughts within,
    Where do we end, and where do we begin?
    Spectral centroid low, a gentle glow,
    Through the quiet night, watch the moments go.

    (Bridge)
    Zero crossings are few, feeling blue,
    But in a peaceful way, me and you.
    No need to rush, in this evening hush,
    Just the sound of the beat, a gentle brush.

    (Chorus)
    Calm and sad, but not really bad,
    Just reflecting on the times we had.
    The tempo's low, letting emotions show,
    In this mellow groove, watch the feelings grow.

    (Outro)
    Yeah, calm and sad, fading out slow,
    Letting the music take control.
    Peaceful end, until we meet again,
    In this calm moment, where the feelings blend.
    """
    print("Using dummy generated_lyrics for demonstration.")


# Load the audio file
try:
    audio_segment = AudioSegment.from_wav(processed_audio_path)
    audio_duration_ms = len(audio_segment)
    print(f"Audio file loaded successfully. Duration: {audio_duration_ms} ms")
except Exception as e:
    print(f"Error loading audio file: {e}")
    audio_segment = None
    audio_duration_ms = 0

# Simple lyric synchronization strategy
# Split lyrics into lines and distribute them evenly over the audio duration.
lyrics_lines = generated_lyrics_to_sync.strip().split('\n')
num_lines = len(lyrics_lines)
line_duration_ms = audio_duration_ms / num_lines if num_lines > 0 else 0
print(f"Splitting lyrics into {num_lines} lines.")
print(f"Estimated duration per line: {line_duration_ms:.2f} ms")


def play_audio(segment):
    """Plays the given pydub AudioSegment."""
    if segment:
        print("Starting audio playback...")
        play(segment)
        print("Audio playback finished.")
    else:
        print("No audio segment to play.")

def display_lyrics(lines, line_duration):
    """Displays lyrics line by line with simulated timing."""
    print("Starting lyric display...")
    for i, line in enumerate(lines):
        print(line)
        if i < len(lines) - 1:
            # Sleep for the estimated duration of the line
            time.sleep(line_duration / 1000.0) # time.sleep expects seconds
    print("Lyric display finished.")

# Ensure playback and lyric display start simultaneously using threads
if audio_segment:
    # Create threads for audio playback and lyric display
    audio_thread = threading.Thread(target=play_audio, args=(audio_segment,))
    lyrics_thread = threading.Thread(target=display_lyrics, args=(lyrics_lines, line_duration_ms))

    # Start both threads
    audio_thread.start()
    lyrics_thread.start()

    # Wait for both threads to complete
    audio_thread.join()
    lyrics_thread.join()

    print("Playback and lyric display completed.")
else:
    print("Cannot start playback and lyric display due to audio loading error.")

## Simple visualizer

### Subtask:
Create a basic soundwave visualizer that reacts to the audio playback.


**Reasoning**:
Import the necessary libraries for creating the soundwave visualizer.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import librosa

**Reasoning**:
Load the audio file, calculate RMS energy, create a time axis, and plot the soundwave visualizer.



In [ ]:
# Assume processed_audio_path and sr are available from previous steps
# If not, define them for demonstration purposes.
if 'processed_audio_path' not in locals():
    processed_audio_path = 'processed_audio/downloaded_audio.wav' # Placeholder

# Load the audio file
try:
    y, sr = librosa.load(processed_audio_path)
    print(f"Successfully loaded audio file from {processed_audio_path}")
except Exception as e:
    print(f"Error loading audio file: {e}")
    y, sr = None, None

if y is not None and sr is not None:
    # Calculate RMS energy
    rms = librosa.feature.rms(y=y)[0]

    # Create a time axis for the RMS values
    times = librosa.times_like(rms, sr=sr)

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(times, rms)

    # Add labels and title
    plt.xlabel("Time (s)")
    plt.ylabel("RMS Energy")
    plt.title("Soundwave Visualizer")

    # Display the plot
    plt.show()
else:
    print("Audio data not available, cannot generate soundwave visualizer.")

## Genre expansion

### Subtask:
Integrate additional genres into the AI lyric generation process.


**Reasoning**:
Update the `create_lyric_prompt` function to include genre-specific instructions and keywords. This involves modifying the existing function to handle a wider variety of genres and incorporate elements that influence lyrical style, structure, and themes based on the selected genre.



In [ ]:
def create_lyric_prompt(audio_features, user_genre, user_theme, explicit_toggle):
    """
    Creates a prompt string for the language model based on audio features and user inputs,
    with enhanced genre-specific guidance.

    Args:
        audio_features: Dictionary containing 'bpm', 'tempo', 'estimated_mood'.
        user_genre: User-specified music genre (e.g., "Hip Hop", "Pop", "Rock", "Blues", "Country", "Electronic").
        user_theme: User-specified theme for the lyrics (e.g., "love", "party", "struggle", "nature", "heartbreak").
        explicit_toggle: Boolean indicating whether explicit lyrics are allowed.

    Returns:
        A formatted prompt string.
    """
    prompt = f"Generate lyrics for a song. "

    # Genre-specific instructions and keywords
    genre_instructions = {
        "Hip Hop": "Focus on rhythm, rhyme schemes (like AABB, ABAB), wordplay, and storytelling. Use urban vocabulary. Include ad-libs.",
        "Pop": "Write catchy melodies and simple, relatable themes. Use a verse-chorus structure. Keep language accessible.",
        "Rock": "Incorporate powerful imagery, emotional themes, and potentially a bridge section. Can be introspective or rebellious.",
        "Blues": "Express feelings of sadness, hardship, or sometimes resilience. Use a typical AAB lyrical structure within verses. Include themes of struggle, love, or loss.",
        "Country": "Tell a story, often set in rural or small-town environments. Themes of love, loss, hard work, and everyday life. Use straightforward language and relatable scenarios.",
        "Electronic": "Lyrics can be repetitive or sparse, focusing on mood and atmosphere. Themes might be abstract, futuristic, or focused on the party/dance experience.",
        "R&B": "Focus on smooth melodies and emotional expression, often about love, relationships, or desire. Use vocal runs and ad-libs.",
        "Reggae": "Incorporate themes of peace, love, social justice, and spirituality. Use a relaxed rhythm and potentially Jamaican Patois influences.",
        "Jazz": "Lyrics can be improvisational or tell complex stories. Focus on mood and atmosphere, often with themes of love, city life, or introspection.",
        "Folk": "Tell a story, often with simple melodies and acoustic instrumentation in mind. Themes of nature, history, social issues, or personal journeys.",
    }

    if user_genre and user_genre in genre_instructions:
        prompt += f"The genre is {user_genre}. {genre_instructions[user_genre]} "
    elif user_genre:
        # Fallback for unknown genres
        prompt += f"The genre is {user_genre}. Write lyrics appropriate for this genre. "

    if user_theme:
        prompt += f"The theme is about {user_theme}. "

    # Incorporate audio features (BPM and Mood are most relevant for lyrical flow and content)
    if audio_features:
        if audio_features.get('bpm') is not None and audio_features['bpm'] > 0:
             # Adjusting prompt based on BPM for rhythm suggestion
             if audio_features['bpm'] < 80:
                 prompt += "The beat is slow, the lyrics should have a relaxed flow and longer phrases. "
             elif audio_features['bpm'] < 120:
                 prompt += "The beat has a moderate pace, the lyrics can have a balanced flow. "
             else:
                 prompt += "The beat is fast and energetic, the lyrics should match the rhythm with shorter, punchier lines. "

        if audio_features.get('estimated_mood'):
            prompt += f"The mood of the music is {audio_features['estimated_mood']}. The lyrics should reflect this mood. "

    # Add explicit toggle consideration (simple instruction for the model)
    if not explicit_toggle:
        prompt += "Avoid explicit language and sensitive topics. "
    else:
        prompt += "Explicit language is allowed. " # Explicitly state if allowed

    prompt += "Write the lyrics now:" # Indicate where the generated lyrics should start

    return prompt

# Example usage with a different genre and theme
user_genre_example = "Blues"
user_theme_example = "hard times"
explicit_toggle_example = False

if audio_analysis_results is not None:
    lyric_prompt_example = create_lyric_prompt(audio_analysis_results, user_genre_example, user_theme_example, explicit_toggle_example)
    print(f"\nGenerated Prompt (Example):\n{lyric_prompt_example}")

    # You would then use the generator with this prompt:
    # if generator is not None:
    #     generated_lyrics_example = generator(lyric_prompt_example, max_length=150, num_return_sequences=1, temperature=0.7)[0]['generated_text']
    #     print("\nGenerated Lyrics (Example):")
    #     print(generated_lyrics_example)
else:
    print("Audio analysis results not available, cannot create example lyric prompt.")

## Visualizer Enhancement

### Subtask: Enhance the soundwave visualizer.

**Reasoning:**
Generate a basic soundwave visualizer as a starting point for enhancement. This code loads the audio, calculates RMS energy, and plots the soundwave.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import os

# Assume processed_audio_path and sr are available from previous steps
# If not, define them for demonstration purposes.
if 'processed_audio_path' not in locals() or not os.path.exists(processed_audio_path):
    processed_audio_path = 'processed_audio/downloaded_audio.wav' # Placeholder
    # Create a dummy file if it doesn't exist for demonstration
    if not os.path.exists('processed_audio'):
        os.makedirs('processed_audio')
    if not os.path.exists(processed_audio_path):
        import soundfile as sf
        samplerate = 22050
        duration = 10
        data = np.zeros(duration * samplerate)
        sf.write(processed_audio_path, data, samplerate)
        print(f"Created a dummy WAV file at: {processed_audio_path}")


# Load the audio file
try:
    y, sr = librosa.load(processed_audio_path)
    print(f"Successfully loaded audio file from {processed_audio_path}")
except Exception as e:
    print(f"Error loading audio file for visualizer: {e}")
    y, sr = None, None

if y is not None and sr is not None:
    # Calculate RMS energy over small frames
    frame_length = 2048 # You can adjust this
    hop_length = 512   # You can adjust this
    rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
    rms_times = librosa.times_like(rms, sr=sr, hop_length=hop_length)

    # Set up the plot
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(rms_times, rms, lw=2)
    ax.set_ylim(0, rms.max() * 1.1)
    ax.set_xlim(0, rms_times.max())
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("RMS Energy")
    ax.set_title("Soundwave Visualizer")
    plt.show()

else:
    print("Audio data not available, cannot generate soundwave visualizer.")

# Task
Create a unified Flask application with a basic frontend structure to handle audio processing, lyric generation, visualization, playback, recording, and export, integrating all the previously developed components into a single web interface.

In [ ]:
!apt-get update
!apt-get install -y libportaudio2

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
!pip install sounddevice numpy scipy

## Unified flask app

### Subtask:
Consolidate the Flask routes and application logic from the different components into a single Flask application instance.


**Reasoning**:
Consolidate the Flask routes and application logic into a single Flask application instance. This involves instantiating the Flask app once, defining all routes using this instance, and ensuring data can be shared between routes. I will combine the relevant code from the previous Flask-related cells into a single cell.



In [ ]:
import os
import yt_dlp
from pydub import AudioSegment
from flask import Flask, request, render_template_string, redirect, url_for, send_file
from werkzeug.utils import secure_filename
import librosa
import numpy as np
import sounddevice as sd
import scipy.io.wavfile as wav
import time
import threading
from transformers import pipeline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import base64 # Import base64 for image encoding
from io import BytesIO # Import BytesIO for image buffer

# Instantiate a single Flask application
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs('processed_audio', exist_ok=True)
os.makedirs('user_recordings', exist_ok=True)
os.makedirs('final_mixes', exist_ok=True)
os.makedirs('visualizer_frames', exist_ok=True) # Directory to save visualizer frames


# Global variables to share data between routes
processed_audio_path = None
audio_analysis_results = None
final_generated_lyrics = None
generator = None # Language model generator instance
audio_duration_s = 0 # To store audio duration for visualization


# Initialize the language model (can be done once at the start)
try:
    # Use a smaller model for potentially faster loading in Colab
    generator = pipeline('text-generation', model='distilgpt2')
    print("Successfully loaded distilgpt2 model for text generation.")
except Exception as e:
    print(f"Error loading language model: {e}")
    generator = None


# Define allowed file types for upload
ALLOWED_EXTENSIONS = {'mp3', 'wav'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def process_audio_input(input_source):
    """
    Processes audio input from either a file upload or a URL.

    Args:
        input_source: Either a file storage object (from Flask request.files) or a string URL.

    Returns:
        The path to the processed WAV file, or None if processing failed.
    """
    output_dir = 'processed_audio'
    os.makedirs(output_dir, exist_ok=True)

    if hasattr(input_source, 'filename'):  # Handle file upload
        if input_source and allowed_file(input_source.filename):
            filename = secure_filename(input_source.filename)
            filepath = os.path.join(output_dir, filename)
            input_source.save(filepath)
            try:
                audio = AudioSegment.from_file(filepath)
                wav_path = os.path.join(output_dir, os.path.splitext(filename)[0] + '.wav')
                audio.export(wav_path, format='wav')
                os.remove(filepath) # Clean up the original uploaded file
                return wav_path
            except Exception as e:
                print(f"Error processing uploaded audio file: {e}")
                os.remove(filepath)
                return None
        else:
            print("Invalid file type or no file uploaded.")
            return None
    elif isinstance(input_source, str): # Handle URL
        url = input_source
        try:
            ydl_opts = {
                'format': 'bestaudio/best',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'wav',
                }],
                'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
                'extract_flat': True,
                'noplaylist': True,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(url, download=False)
                if 'entries' in info_dict: # Handle playlists, although we set noplaylist=True, some URLs might be interpreted as playlists
                    info_dict = info_dict['entries'][0] # Take the first video
                title = info_dict.get('title', 'downloaded_audio')
                output_filename = os.path.join(output_dir, f"{title}.wav")
                ydl.download([url])
                return output_filename
        except Exception as e:
            print(f"Error downloading audio from URL: {e}")
            return None
    else:
        print("Invalid input source.")
        return None

def analyze_audio(audio_path):
    """Analyzes audio to detect BPM, tempo, and mood."""
    global audio_duration_s
    if not os.path.exists(audio_path):
        print(f"Audio file not found for analysis: {audio_path}")
        return None

    try:
        y, sr = librosa.load(audio_path)
        audio_duration_s = librosa.get_duration(y=y, sr=sr)
        print(f"Successfully loaded audio file for analysis: {audio_path}")
        print(f"Audio duration: {audio_duration_s:.2f} seconds")


        # Calculate BPM and tempo
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        print(f"Estimated BPM: {tempo:.2f}")

        # Simple heuristic for mood (highly simplified for demonstration)
        spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        zcr = librosa.feature.zero_crossing_rate(y)[0]
        mean_centroid = np.mean(spectral_centroids)
        mean_zcr = np.mean(zcr)

        estimated_mood = "Neutral"
        if tempo > 120 and mean_centroid > 2000:
            estimated_mood = "Energetic/Happy"
        elif tempo < 80 and mean_centroid < 1500:
             estimated_mood = "Calm/Sad"
        elif tempo > 140 and mean_zcr > 0.1:
             estimated_mood = "Fast/Intense"

        print(f"Estimated Mood (Simplified Heuristic): {estimated_mood}")

        return {
            'bpm': tempo,
            'tempo': tempo,
            'estimated_mood': estimated_mood,
            'duration': audio_duration_s,
            'y': y, # Store audio time series data for visualizer
            'sr': sr  # Store sampling rate for visualizer
        }

    except Exception as e:
        print(f"Error during audio analysis: {e}")
        audio_duration_s = 0
        return None

def create_lyric_prompt(audio_features, user_genre, user_theme, explicit_toggle):
    """
    Creates a prompt string for the language model based on audio features and user inputs,
    with enhanced genre-specific guidance.
    """
    prompt = f"Generate lyrics for a song. "

    genre_instructions = {
        "Hip Hop": "Focus on rhythm, rhyme schemes (like AABB, ABAB), wordplay, and storytelling. Use urban vocabulary. Include ad-libs.",
        "Pop": "Write catchy melodies and simple, relatable themes. Use a verse-chorus structure. Keep language accessible.",
        "Rock": "Incorporate powerful imagery, emotional themes, and potentially a bridge section. Can be introspective or rebellious.",
        "Blues": "Express feelings of sadness, hardship, or sometimes resilience. Use a typical AAB lyrical structure within verses. Include themes of struggle, love, or loss.",
        "Country": "Tell a story, often set in rural or small-town environments. Themes of love, loss, hard work, and everyday life. Use straightforward language and relatable scenarios.",
        "Electronic": "Lyrics can be repetitive or sparse, focusing on mood and atmosphere. Themes might be abstract, futuristic, or focused on the party/dance experience.",
        "R&B": "Focus on smooth melodies and emotional expression, often about love, relationships, or desire. Use vocal runs and ad-libs.",
        "Reggae": "Incorporate themes of peace, love, social justice, and spirituality. Use a relaxed rhythm and potentially Jamaican Patois influences.",
        "Jazz": "Lyrics can be improvisational or tell complex stories. Focus on mood and atmosphere, often with themes of love, city life, or introspection.",
        "Folk": "Tell a story, often with simple melodies and acoustic instrumentation in mind. Themes of nature, history, social issues, or personal journeys.",
    }

    if user_genre and user_genre in genre_instructions:
        prompt += f"The genre is {user_genre}. {genre_instructions[user_genre]} "
    elif user_genre:
        prompt += f"The genre is {user_genre}. Write lyrics appropriate for this genre. "

    if user_theme:
        prompt += f"The theme is about {user_theme}. "

    if audio_features:
        if audio_features.get('bpm') is not None and audio_features['bpm'] > 0:
             if audio_features['bpm'] < 80:
                 prompt += "The beat is slow, the lyrics should have a relaxed flow and longer phrases. "
             elif audio_features['bpm'] < 120:
                 prompt += "The beat has a moderate pace, the lyrics can have a balanced flow. "
             else:
                 prompt += "The beat is fast and energetic, the lyrics should match the rhythm with shorter, punchier lines. "

        if audio_features.get('estimated_mood'):
            prompt += f"The mood of the music is {audio_features['estimated_mood']}. The lyrics should reflect this mood. "

    if not explicit_toggle:
        prompt += "Avoid explicit language and sensitive topics. "
    else:
        prompt += "Explicit language is allowed. "

    prompt += "Write the lyrics now:"

    return prompt

def generate_lyrics(audio_features, user_genre, user_theme, explicit_toggle):
    """Generates lyrics using the language model."""
    global generator
    if generator is None:
        print("Language model not loaded.")
        return "Error: Language model not available."

    if audio_features is None:
        print("Audio analysis results not available.")
        return "Error: Audio analysis results not available."

    prompt = create_lyric_prompt(audio_features, user_genre, user_theme, explicit_toggle)

    try:
        generated_text = generator(
            prompt,
            max_length=500,
            num_return_sequences=1,
            temperature=0.9,
            top_k=50,
            top_p=0.95,
            truncation=True
        )[0]['generated_text']

        # Attempt to remove the prompt from the generated text
        if generated_text.startswith(prompt):
            generated_text = generated_text[len(prompt):].strip()

        return generated_text

    except Exception as e:
        print(f"Error during lyric generation: {e}")
        return f"Error generating lyrics: {e}"

# Note: Recording audio directly in Flask server is not typical or recommended.
# This function is a placeholder/demonstration. Actual recording would be client-side.
def record_audio_dummy(filename, duration):
    """Creates a dummy recording file."""
    print(f"Creating a dummy recording file at {filename} for {duration} seconds.")
    try:
        samplerate = 44100
        data = np.random.rand(int(duration * samplerate)) * 0.1 # Little bit of noise
        wav.write(filename, samplerate, data)
        print(f"Dummy recording saved to {filename}")
        return filename
    except Exception as e:
        print(f"Error creating dummy recording: {e}")
        return None


def combine_audio(beat_path, vocals_path):
    """Combines beat audio with recorded vocals."""
    if not os.path.exists(beat_path):
        return f"Error: Beat audio file not found at {beat_path}"
    if not os.path.exists(vocals_path):
        return f"Error: Vocal audio file not found at {vocals_path}"

    try:
        beat_audio = AudioSegment.from_wav(beat_path)
        vocal_audio = AudioSegment.from_wav(vocals_path)

        # Ensure same frame rate
        if beat_audio.frame_rate != vocal_audio.frame_rate:
            print(f"Warning: Mismatch in frame rates. Resampling vocal audio from {vocal_audio.frame_rate} to {beat_audio.frame_rate}")
            vocal_audio = vocal_audio.set_frame_rate(beat_audio.frame_rate)

        # Ensure similar length
        min_duration = min(len(beat_audio), len(vocal_audio))
        beat_audio = beat_audio[:min_duration]
        vocal_audio = vocal_audio[:min_duration]

        # Overlay
        combined_audio = beat_audio.overlay(vocal_audio)

        export_dir = 'final_mixes'
        os.makedirs(export_dir, exist_ok=True)
        final_output_path = os.path.join(export_dir, 'final_performance_mix.wav')
        combined_audio.export(final_output_path, format="wav")

        return final_output_path

    except Exception as e:
        print(f"Error combining audio files: {e}")
        return f"Error combining audio files: {e}"

def create_soundwave_visualizer(y, sr, audio_duration_s):
    """Generates a static soundwave visualizer image."""
    if y is None or sr is None:
        return None

    try:
        # Calculate RMS energy over small frames
        frame_length = 2048 # You can adjust this
        hop_length = 512   # You can adjust this
        rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
        rms_times = librosa.times_like(rms, sr=sr, hop_length=hop_length)

        # Set up the plot
        fig, ax = plt.subplots(figsize=(12, 4))
        ax.plot(rms_times, rms, lw=2)
        ax.set_ylim(0, rms.max() * 1.1)
        ax.set_xlim(0, audio_duration_s)
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("RMS Energy")
        ax.set_title("Soundwave Visualizer")

        # Save plot to a bytes buffer
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        plt.close(fig) # Close the figure to free memory

        # Encode image to base64
        image_base64 = base64.b64encode(buf.read()).decode('ascii')

        return image_base64

    except Exception as e:
        print(f"Error generating soundwave visualizer: {e}")
        return None


# Flask Routes

@app.route('/', methods=['GET', 'POST'])
def index():
    global processed_audio_path, audio_analysis_results, final_generated_lyrics, audio_duration_s
    processed_file = None
    analysis_results_display = None
    generated_lyrics_display = None
    visualizer_image_base64 = None

    if request.method == 'POST':
        if 'audio_file' in request.files and request.files['audio_file'].filename != '':
            audio_file = request.files['audio_file']
            processed_file = process_audio_input(audio_file)
        elif 'audio_url' in request.form and request.form['audio_url'] != '':
            audio_url = request.form['audio_url']
            processed_file = process_audio_input(audio_url)
        elif 'genre' in request.form: # Handle lyric generation request
             user_genre = request.form.get('genre', 'Hip Hop')
             user_theme = request.form.get('theme', 'general')
             explicit_toggle = request.form.get('explicit', 'off') == 'on'
             if audio_analysis_results:
                 final_generated_lyrics = generate_lyrics(audio_analysis_results, user_genre, user_theme, explicit_toggle)
                 generated_lyrics_display = final_generated_lyrics
             else:
                 generated_lyrics_display = "Please upload/process audio first to generate lyrics."

        if processed_file:
            processed_audio_path = processed_file
            audio_analysis_results = analyze_audio(processed_audio_path)
            if audio_analysis_results:
                analysis_results_display = f"BPM: {audio_analysis_results['bpm']:.2f}, Mood: {audio_analysis_results['estimated_mood']}, Duration: {audio_analysis_results['duration']:.2f} seconds"
                # Generate visualizer image immediately after analysis
                visualizer_image_base64 = create_soundwave_visualizer(audio_analysis_results.get('y'), audio_analysis_results.get('sr'), audio_analysis_results.get('duration', 0))


    # Display existing data on GET requests or after POST
    if audio_analysis_results:
         analysis_results_display = f"BPM: {audio_analysis_results['bpm']:.2f}, Mood: {audio_analysis_results['estimated_mood']}, Duration: {audio_analysis_results['duration']:.2f} seconds"
         # Regenerate visualizer on page load if analysis results exist
         visualizer_image_base64 = create_soundwave_visualizer(audio_analysis_results.get('y'), audio_analysis_results.get('sr'), audio_analysis_results.get('duration', 0))

    if final_generated_lyrics:
        generated_lyrics_display = final_generated_lyrics


    return render_template_string('''
    <!doctype html>
    <title>Ai Lyrical Arsonist</title>
    <h1>Ai Lyrical Arsonist</h1>

    <h2>Upload an audio file or provide a URL</h2>
    <form method=post enctype=multipart/form-data>
      <p><input type=file name=audio_file>
         <input type=submit value=Upload File>
    </form>
    <form method=post>
      <p><input type=text name=audio_url placeholder="Enter URL">
         <input type=submit value=Download URL>
    </form>

    {% if processed_audio_path %}
    <h2>Processed Audio:</h2>
    <p>File ready for analysis and lyric generation.</p>
    {% endif %}

    {% if analysis_results_display %}
    <h2>Audio Analysis Results:</h2>
    <p>{{ analysis_results_display }}</p>
    {% endif %}

    {% if visualizer_image_base64 %}
    <h2>Soundwave Visualizer:</h2>
    <img src="data:image/png;base64,{{ visualizer_image_base64 }}" alt="Soundwave Visualizer">
    {% endif %}


    <h2>Generate Lyrics</h2>
    <form method=post>
      <p>Genre: <input type=text name=genre value="Hip Hop"></p>
      <p>Theme: <input type=text name=theme value="general"></p>
      <p>Explicit: <input type=checkbox name=explicit value="on"></p>
      <input type=submit value=Generate Lyrics>
    </form>

    {% if generated_lyrics_display %}
    <h2>Generated Lyrics:</h2>
    <pre>{{ generated_lyrics_display }}</pre>
    <p><a href="{{ url_for('edit_lyrics') }}">Edit Lyrics</a></p>
    {% endif %}

    {% if processed_audio_path and generated_lyrics_display %}
    <h2>Playback and Recording</h2>
    <p><a href="{{ url_for('playback') }}">Start Playback and Sync (Text-based simulation)</a></p>
     <p><a href="{{ url_for('record_page') }}">Record Performance (Client-side required for real app)</a></p>
    {% endif %}

    {% if processed_audio_path %}
        {% set dummy_recording_path = 'user_recordings/user_performance.wav' %}
        {% if os.path.exists(dummy_recording_path) %}
            <h2>Combine Audio</h2>
            <p><a href="{{ url_for('combine') }}">Combine Beat and Vocals</a></p>
        {% else %}
             <h2>Record Performance</h2>
             <p>Record your performance first to enable audio combining.</p>
              <p><a href="{{ url_for('record_page') }}">Record Performance (Client-side required for real app)</a></p>
        {% endif %}
    {% endif %}


    {% if os.path.exists('final_mixes/final_performance_mix.wav') %}
    <h2>Final Mix</h2>
    <p><a href="{{ url_for('download_mix') }}">Download Final Mix</a></p>
    {% endif %}


    ''', processed_audio_path=processed_audio_path,
         analysis_results_display=analysis_analysis_results,
         generated_lyrics_display=generated_lyrics_display,
         visualizer_image_base64=visualizer_image_base64,
         os=os) # Pass os to check file existence

@app.route('/edit_lyrics', methods=['GET', 'POST'])
def edit_lyrics():
    global final_generated_lyrics
    if request.method == 'POST':
        final_generated_lyrics = request.form.get('lyrics_textarea', '')
        print("Lyrics edited and saved.")
        return redirect(url_for('index')) # Redirect back to the main page

    return render_template_string('''
    <!doctype html>
    <title>Edit Lyrics</title>
    <h1>Edit Your Lyrics</h1>
    <form method=post>
      <textarea name="lyrics_textarea" rows="20" cols="80">{{ lyrics }}</textarea><br><br>
      <input type=submit value=Save>
    </form>
    <p><a href="{{ url_for('index') }}">Back to Home</a></p>
    ''', lyrics=final_generated_lyrics if final_generated_lyrics else "No lyrics generated yet.")


@app.route('/playback')
def playback():
    global processed_audio_path, final_generated_lyrics

    if not processed_audio_path or not os.path.exists(processed_audio_path):
        return "Error: Processed audio not available.", 400
    if not final_generated_lyrics:
        return "Error: Lyrics not generated yet.", 400

    # Simple lyric synchronization strategy (as before)
    lyrics_lines = final_generated_lyrics.strip().split('\n')
    try:
        audio_segment = AudioSegment.from_wav(processed_audio_path)
        audio_duration_ms = len(audio_segment)
        num_lines = len(lyrics_lines)
        line_duration_ms = audio_duration_ms / num_lines if num_lines > 0 else 0
    except Exception as e:
        return f"Error loading audio for playback: {e}", 500

    # This is a simplified text-based playback simulation for the notebook environment.
    # A real web app would require client-side JavaScript for synchronized audio and lyric display.
    playback_output = "Starting playback and lyric sync...\n"
    for i, line in enumerate(lyrics_lines):
        playback_output += line + "\n"
        if i < len(lyrics_lines) - 1:
             # Simulate timing (not real-time in a Flask response)
             pass # In a real app, this timing is handled client-side

    playback_output += "Playback and lyric display completed.\n"

    return render_template_string('''
    <!doctype html>
    <title>Playback and Sync</title>
    <h1>Playback and Lyric Sync (Text-based simulation)</h1>
    <pre>{{ output }}</pre>
    <p><a href="{{ url_for('index') }}">Back to Home</a></p>
    ''', output=playback_output)


@app.route('/record_page')
def record_page():
     # This route provides a page with instructions for recording.
     # Actual recording is client-side in a real web app.
     # For this demo, we'll create a dummy recording file when the user clicks 'Record Dummy'.
     return render_template_string('''
     <!doctype html>
     <title>Record Performance</title>
     <h1>Record Your Performance</h1>
     <p>In a real web application, this page would use your microphone to record your vocals over the beat.</p>
     <p>For this demonstration, you can create a dummy recording file to proceed with the combining step.</p>
     <p><a href="{{ url_for('create_dummy_recording') }}">Create Dummy Recording File</a></p>
     <p><a href="{{ url_for('index') }}">Back to Home</a></p>
     ''')

@app.route('/create_dummy_recording')
def create_dummy_recording():
    global audio_duration_s
    dummy_recording_path = 'user_recordings/user_performance.wav'
    # Create a dummy recording with the same duration as the processed audio
    record_audio_dummy(dummy_recording_path, audio_duration_s if audio_duration_s > 0 else 10) # Default to 10 secs if duration is 0
    return redirect(url_for('index'))


@app.route('/combine')
def combine():
    global processed_audio_path
    user_recording_path = 'user_recordings/user_performance.wav' # Assume this is the default recording file

    if not processed_audio_path or not os.path.exists(processed_audio_path):
        return "Error: Processed beat audio not available.", 400
    if not os.path.exists(user_recording_path):
        return f"Error: User recording not found at {user_recording_path}. Please ensure a recording has been made (or a dummy file exists).", 400

    final_mix_path = combine_audio(processed_audio_path, user_recording_path)

    if final_mix_path.startswith("Error"):
        return final_mix_path, 500
    else:
        return render_template_string('''
        <!doctype html>
        <title>Combine Audio</title>
        <h1>Audio Combined!</h1>
        <p>Final mix saved to: {{ final_mix_path }}</p>
        <p><a href="{{ url_for('download_mix') }}">Download Final Mix</a></p>
        <p><a href="{{ url_for('index') }}">Back to Home</a></p>
        ''', final_mix_path=final_mix_path)

@app.route('/download_mix')
def download_mix():
    final_mix_path = 'final_mixes/final_performance_mix.wav'
    if os.path.exists(final_mix_path):
        return send_file(final_mix_path, as_attachment=True)
    else:
        return "Error: Final mix file not found.", 404


# To run the Flask app (for demonstration purposes)
# Note: In a Colab environment, you might need to use ngrok or a similar service
# to expose your local Flask server to the internet for testing in a browser.
# For simple testing within Colab, you can use the built-in preview.

if __name__ == '__main__':
    # This will run the Flask development server
    # debug=True allows for automatic reloading and helpful error pages
    # Set host='0.0.0.0' to make the server accessible externally if needed (e.g., for ngrok)
    app.run(debug=True, host='0.0.0.0', port=5000)

In [ ]:
# Run the Flask app
# This cell will start the Flask development server.
# Note: In a Colab environment, you might need to use ngrok or a similar service
# to expose your local Flask server to the internet for testing in a browser.
# For simple testing within Colab, you can use the built-in preview.

if __name__ == '__main__':
    # This will run the Flask development server
    # Set host='0.0.0.0' to make the server accessible externally if needed (e.g., for ngrok)
    app.run(debug=True, host='0.0.0.0', port=5000)